In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import gzip
from tensorflow import keras
import pandas as pd
import seaborn as sns

import os
os.chdir('..')
from training_utils import *
os.chdir("./parameter_study")

In [ ]:
dataset = load_datasets("../datasets")
figure_dir = "plots"
fmt = "pdf"
with gzip.open("para.pkl.gz","r+b") as f:
        dic = pickle.load(f)

In [ ]:

x_dict = {4:0, 8:1, 16:2, 32:3}
x = [4,8,16,32]
y_tanh = np.zeros((4,4))
y_relu = np.zeros((4,4))

for k,v in dic["para1"]["history"].items():
    paras = k.split("_")
    arr = y_tanh
    if paras[0] == "relu":
        arr = y_relu
    l = int(paras[1])
    n = int(paras[2])
    arr[l-1, x_dict[n]] = np.min(v.history["loss"])

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, sharex=True,subplot_kw=dict(box_aspect=1, xlabel="# of neurons per layer", xticks=x, yscale="log"))
markers = ["d", "o", "^", "s"]
ax1.set_title("tanh")
for i in range(len(y_tanh)):
    ax1.plot(x, y_tanh[i], marker=markers[i], label=f"{i+1}")

ax1.grid(axis='y')
ax1.set_ylabel("MSE loss")
ax1.set_axisbelow(True)


ax2.set_title("ReLU")
for i in range(len(y_relu)):
    plt.plot(x, y_relu[i], marker=markers[i], label=f"{i+1}")
ax2.legend(title="Layer(s)")
ax2.grid(axis='y')
ax2.set_axisbelow(True)


fig.tight_layout()
fig.savefig(f"{figure_dir}/para1_net.{fmt}", bbox_inches="tight")

plt.show()

In [ ]:
para1 = dic["para1"]
model1 = para1["models"]["relu_2_8"]
X_norms1 = para1["X_norms"]
Y_norms1 = para1["Y_norms"]

value_dict = {"4-45":"tab:blue", "4-HV":"tab:red", "1-HV":"tab:green", "1-22p5":"tab:orange"}
df1 = evaluate_model(model1, dataset, X_norms1, Y_norms1)

In [ ]:
ax1 = sns.scatterplot(data=df1, y="MSE", x="Material_Amplitude", hue="Plate", palette=value_dict, style="Mesh size")
ax1.set_axisbelow(True)
ax1.grid(axis='y')
ax1.semilogy()
ax1.set_title("relu_2_8 trained on one simulation")
plt.savefig(f"{figure_dir}/para1_all.{fmt}",bbox_inches="tight")
plt.show()

In [ ]:
fig, ((ax1,ax2), (ax3,ax4)) = plt.subplots(2, 2, figsize=(7,7),sharey=True, sharex=True,subplot_kw=dict(box_aspect=1))
axs = [ax1,ax2,ax3,ax4]
names = ["1-HV_3p0_T6_15","4-HV_1p5_T4_15","4-45_3p0_T4_10","4-45_1p5_T7_10"]

for name,ax,e in zip(names,axs,range(4)):
    db = dataset[name]
    PLAG = db["PLAG"]
    P = db["P"]
    N = db["N"]
    labels = db["labels"]
    lagp = np.ones_like(P)
    for i in range(len(lagp)):
        lagp[i] *= PLAG[i,1]

    lagp /= X_norms1[0]
    N /= X_norms1[1]
    P /= Y_norms1[0]

    nsteps = P.shape[0]
    nels = P.shape[1]
    errors = np.zeros_like(labels)
    X = np.transpose([lagp , N])
    Y = np.transpose([P])

    Y_shape = Y.shape
    prediction = model1.predict(X.reshape((Y_shape[0]*Y_shape[1],2)), batch_size=Y_shape[0]*Y_shape[1]).reshape(Y_shape)
    error = Y - prediction

    errors = np.zeros(nels)
    for i in range(nels):
        errors[i] = np.dot(error[i,:,0], error[i,:,0])/nels
    eid = errors.argmax()

    ax.set_title(f"{name} " + f"(~1E-0{6-e})")
    ax.plot(PLAG[:,0]*1000, P[:,eid]*Y_norms1[0], label="Loading pressure")
    ax.plot(PLAG[:,0]*1000, prediction[eid,:,0]*Y_norms1[0], label="Predicted pressure")
    #plt.plot(PLAG[:,0],N[:,eid], label="N")
    #ax.legend()
    ax.grid(axis="both")
    ax.set_xlabel("Time [ms]")
    ax.set_ylabel("Pressure [MPa]")
    ax.label_outer()

fig.legend(labels=["Loading pressure", "Predicted pressure"], loc='center', bbox_to_anchor=(0.5, -0.02), ncol=2)

fig.tight_layout()  
fig.savefig(f"{figure_dir}/para1_test.{fmt}",bbox_inches="tight")
plt.show()

In [ ]:
para2 = dic["para2"]
model2 = para2["model"]
X_norms2 = para2["X_norms"]
Y_norms2 = para2["Y_norms"]

df2 = evaluate_model(model2, dataset, X_norms2, Y_norms2)

In [ ]:
ax = sns.scatterplot(data=df2, y="MSE", x="Material_Amplitude", hue="Plate", palette=value_dict, style="Mesh size")

ax.set_axisbelow(True)
ax.grid(axis='y')
ax.semilogy()
ax.set_title("relu_2_8 trained on two simulations")
plt.savefig(f"{figure_dir}/para2_all.{fmt}",bbox_inches="tight")
plt.show()

In [ ]:
para3 = dic["para3"]
model3 = para3["model"]
X_norms3 = para3["X_norms"]
Y_norms3 = para3["Y_norms"]

df3 = evaluate_model(model3, dataset, X_norms3, Y_norms3)

In [ ]:
ax = sns.scatterplot(data=df3, y="MSE", x="Material_Amplitude", hue="Plate", palette=value_dict, style="Mesh size")

ax.set_axisbelow(True)
ax.grid(axis='y')
ax.semilogy()
ax.set_title("relu_2_8 trained on three simulations")
plt.savefig(f"{figure_dir}/para3_all.{fmt}",bbox_inches="tight")
plt.show()

In [ ]:
fig, ((ax1,ax2), (ax3,ax4)) = plt.subplots(2, 2, figsize=(7,7),sharey=True, sharex=True,subplot_kw=dict(box_aspect=1))
axs = [ax1,ax2,ax3,ax4]
names = ["1-HV_3p0_T6_15","4-HV_1p5_T4_15","4-45_3p0_T4_10","4-45_1p5_T7_10"]

for name,ax,e in zip(names,axs,range(4)):
    db = dataset[name]
    PLAG = db["PLAG"]
    P = db["P"]
    N = db["N"]
    labels = db["labels"]
    lagp = np.ones_like(P)
    for i in range(len(lagp)):
        lagp[i] *= PLAG[i,1]

    lagp /= X_norms3[0]
    N /= X_norms3[1]
    P /= Y_norms3[0]

    nsteps = P.shape[0]
    nels = P.shape[1]
    errors = np.zeros_like(labels)
    X = np.transpose([lagp , N])
    Y = np.transpose([P])

    Y_shape = Y.shape
    prediction = model3.predict(X.reshape((Y_shape[0]*Y_shape[1],2)), batch_size=Y_shape[0]*Y_shape[1]).reshape(Y_shape)
    error = Y - prediction

    errors = np.zeros(nels)
    for i in range(nels):
        errors[i] = np.dot(error[i,:,0], error[i,:,0])/nels
    eid = errors.argmax()
    val = df3[df3["Job"]==name]["MSE"].max()
    ax.set_title(f"{name} ~" + f"({val:.0E})")
    ax.plot(PLAG[:,0]*1000, P[:,eid]*Y_norms3[0], label="Loading pressure")
    ax.plot(PLAG[:,0]*1000, prediction[eid,:,0]*Y_norms3[0], label="Predicted pressure")
    #plt.plot(PLAG[:,0],N[:,eid], label="N")
    #ax.legend()
    ax.grid(axis="both")
    ax.set_xlabel("Time [ms]")
    ax.set_ylabel("Pressure [MPa]")
    ax.label_outer()
fig.legend(labels=["Loading pressure", "Predicted pressure"], loc='center', bbox_to_anchor=(0.5, -0.02), ncol=2)

fig.tight_layout()  
fig.savefig(f"{figure_dir}/para3_test.{fmt}",bbox_inches="tight")
plt.show()

In [ ]:
para4= dic["para4"]
model4 = para4["model"]
X_norms4 = para4["X_norms"]
Y_norms4 = para4["Y_norms"]

df4 = evaluate_model(model4, dataset, X_norms4, Y_norms4)

In [ ]:
ax = sns.scatterplot(data=df4, y="MSE", x="Material_Amplitude", hue="Plate", palette=value_dict, style="Mesh size")

ax.set_axisbelow(True)
ax.grid(axis='y')
ax.semilogy()
ax.set_title("relu_2_8 trained on all simulations")
plt.savefig(f"{figure_dir}/para4_all.{fmt}",bbox_inches="tight")
plt.show()

In [ ]:
fig, ((ax1,ax2), (ax3,ax4)) = plt.subplots(2, 2, figsize=(7,7),sharey=True, sharex=True,subplot_kw=dict(box_aspect=1))
axs = [ax1,ax2,ax3,ax4]
names = ["1-HV_3p0_T6_15","4-HV_1p5_T4_15","4-45_3p0_T4_10","4-45_1p5_T7_10"]

for name,ax,e in zip(names,axs,range(4)):
    db = dataset[name]
    PLAG = db["PLAG"]
    P = db["P"]
    N = db["N"]
    labels = db["labels"]
    lagp = np.ones_like(P)
    for i in range(len(lagp)):
        lagp[i] *= PLAG[i,1]

    lagp /= X_norms4[0]
    N /= X_norms4[1]
    P /= Y_norms4[0]

    nsteps = P.shape[0]
    nels = P.shape[1]
    errors = np.zeros_like(labels)
    X = np.transpose([lagp , N])
    Y = np.transpose([P])

    Y_shape = Y.shape
    prediction = model4.predict(X.reshape((Y_shape[0]*Y_shape[1],2)), batch_size=Y_shape[0]*Y_shape[1]).reshape(Y_shape)
    error = Y - prediction

    errors = np.zeros(nels)
    for i in range(nels):
        errors[i] = np.dot(error[i,:,0], error[i,:,0])/nels
    eid = errors.argmax()

    val = df4[df4["Job"]==name]["MSE"].max()
    ax.set_title(f"{name} ~" + f"({val:.0E})")
    ax.plot(PLAG[:,0]*1000, P[:,eid]*Y_norms4[0], label="Loading pressure")
    ax.plot(PLAG[:,0]*1000, prediction[eid,:,0]*Y_norms4[0], label="Predicted pressure")
    #plt.plot(PLAG[:,0],N[:,eid], label="N")
    #ax.legend()
    ax.grid(axis="both")
    ax.set_xlabel("Time [ms]")
    ax.set_ylabel("Pressure [MPa]")
    ax.label_outer()
fig.legend(labels=["Loading pressure", "Predicted pressure"], loc='center', bbox_to_anchor=(0.5, -0.02), ncol=2)

fig.tight_layout()  
fig.savefig(f"{figure_dir}/para4_test.{fmt}",bbox_inches="tight")
plt.show()

In [ ]:
fig, ((ax1,ax2), (ax3,ax4)) = plt.subplots(2, 2, figsize=(7,7),sharey=True, sharex=True,subplot_kw=dict(box_aspect=1, yscale="log"))
sns.scatterplot(ax=ax1, data=df1, y="MSE", x="Material_Amplitude", hue="Plate", palette=value_dict,style="Mesh size", legend=None)
sns.scatterplot(ax=ax2, data=df2, y="MSE", x="Material_Amplitude", hue="Plate", palette=value_dict,style="Mesh size", legend=None)
sns.scatterplot(ax=ax3, data=df3, y="MSE", x="Material_Amplitude", hue="Plate", palette=value_dict,style="Mesh size", legend=None)
sns.scatterplot(ax=ax4, data=df4, y="MSE", x="Material_Amplitude", hue="Plate", palette=value_dict,style="Mesh size")

axs = [ax1,ax2,ax3,ax4]
titles = ["One simulation", "Two simulations", "Three simulations", "All simulations"]

for ax,title in zip(axs,titles):
    ax.set_axisbelow(True)
    ax.grid(axis='y')
    ax.set_title(title)

fig.tight_layout()
fig.savefig(f"{figure_dir}/para_all.{fmt}",bbox_inches="tight")
plt.show()

In [ ]:
plags = np.zeros((48,2))
ps = np.zeros((48,2))
ns = np.zeros((48,2))
jobnames = list()

for i,(jobname,job) in enumerate(dataset.items()):
    jobnames.append(jobname)
    plags[i] = [job["PLAG"][:,1].min(), job["PLAG"][:,1].max()]
    ps[i] = [job["P"].min(), job["P"].max()]
    ns[i] = [job["N"].min(), job["N"].max()]

variables = ["Lagrangian pressure", "Normal (z-component)", "Pressure"]
values = [plags,ns, ps]
ylabels = ["Pressure [MPa]", "Normal", "Pressure [MPa]"]
fig, ((ax1),(ax2), (ax3)) = plt.subplots(3,1, figsize=(6,8), sharex=True)
axs = [ax1,ax2,ax3]

for va,val, ax, ylabel in zip(variables,values,axs,ylabels):

    ax.set_title(va)
    ax.plot(val[:,0], label="Min")
    ax.plot(val[:,1], label="Max")
    ax.set_xlabel("Simulation #")
    ax.set_ylabel(ylabel)
    ax.label_outer()
    ax.grid()

fig.tight_layout()
fig.legend(labels=["Min", "Max"], loc='center', bbox_to_anchor=(0.5, -0.02), ncol=2)
fig.savefig(f"{figure_dir}/vars.{fmt}", bbox_inches="tight")
plt.show()
print(np.array(jobnames)[ns[:,0].argsort()[:20]])

In [ ]:
para5 = dic["para5"]
model5 = para5["model"]
X_norms5 = para5["X_norms"]
Y_norms5 = para5["Y_norms"]

df5 = evaluate_model(model5, dataset, para5["X_norms"], para5["Y_norms"])

In [ ]:
ax = sns.scatterplot(data=df5, y="MSE", x="Material_Amplitude", hue="Plate", palette=value_dict, style="Mesh size")

ax.set_axisbelow(True)
ax.grid(axis='y')
ax.semilogy()
ax.set_title("relu_2_8 trained on three simulations (physical)")
plt.savefig(f"{figure_dir}/para5_all.{fmt}", bbox_inches="tight")
plt.show()

In [ ]:
fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(7,4),sharey=True, sharex=True,subplot_kw=dict(box_aspect=1, yscale="log"))
ax1 = sns.scatterplot(ax=ax1,data=df3, y="MSE", x="Material_Amplitude", hue="Plate", palette=value_dict,style="Mesh size", legend=None)
ax2 = sns.scatterplot(ax=ax2,data=df5, y="MSE", x="Material_Amplitude", hue="Plate", palette=value_dict,style="Mesh size")

axs = [ax1,ax2]
titles = ["iterative / mathematical", "physical"]

for ax,title in zip(axs,titles):
    ax.set_axisbelow(True)
    ax.grid(axis='y')
    ax.set_title(title)
fig.tight_layout()

fig.savefig(f"{figure_dir}/para3_vs_para5.{fmt}", bbox_inches="tight")
plt.show()

In [ ]:
dat10 = dataset["1-22p5_1p5_T4_10"]["PLAG"]
dat15 = dataset["1-22p5_1p5_T4_15"]["PLAG"]

fig = plt.figure(figsize=(4,4))
plt.title("Lagrangian pressure histories")
plt.plot(dat15[:,0]*1000, dat15[:,1], label="15 bar")
plt.plot(dat10[:,0]*1000, dat10[:,1], label="10 bar")
plt.legend(title="Driver pressure")
plt.xlabel("Time [ms]")
plt.ylabel("Pressure [MPa]")
fig.savefig(f"{figure_dir}/pressures.{fmt}", bbox_inches="tight")
plt.show(fig)